In [ ]:
    # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### import libs


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as k

import csv
import cv2
import os

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import VGG16,VGG19
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow import keras


from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split

from scipy import ndimage

In [2]:

img = cv2.imread('../input/emotion-detection-fer/train/angry/im1003.png')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

float_gray = img.astype(np.float32)/255

blur = cv2.GaussianBlur(float_gray, (0, 0), sigmaX=2, sigmaY=2)
num = float_gray - blur

blur = cv2.GaussianBlur(num*num, (0, 0), sigmaX=20, sigmaY=20)
den = cv2.pow(blur, 0.25)

gray = num / den

# cv2.normalize(gray, dst=gray, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX)
gray = cv2.resize(gray,(48,48))

plt.imshow(gray)
print(gray.shape)


In [3]:
original_img =cv2.imread("../input/emotion-detection-fer/train/angry/im1003.png")
original_img = cv2.resize(original_img,(128,128))
plt.imshow(original_img)


In [4]:
imagetype= os.listdir("../input/emotion-detection-fer/train")
print(imagetype)

In [5]:
import numpy
import scipy
import scipy.misc
from PIL import Image



def global_contrast_normalization(filename, s, lmda, epsilon):
    X = cv2.imread(filename)
    
    # replacement for the loop
    X_average = numpy.mean(X)
    print('Mean: ', X_average)
    X = X - X_average

    # `su` is here the mean, instead of the sum
    contrast = numpy.sqrt(lmda + numpy.mean(X**2))

    X = s * X / max(contrast, epsilon)
    plt.imshow(X*255)
    # scipy can handle it
    X= numpy.array(X)
    print(X.shape)

global_contrast_normalization("../input/emotion-detection-fer/train/angry/im1003.png", 1, 10, 0.000000001)



In [12]:
images = []    #images list will store all images
path = "../input/emotion-detection-fer/train"
labels= []

In [13]:
for i in imagetype:
    print(i)
    flower_path=path+"/"+str(i)
    data_list=[j for j in os.listdir(flower_path) if j.endswith(".png") ]
    counter=0
    for data in data_list:
        if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         images.append(img)
#         labels.append(i)
        float_gray = img.astype(np.float32)/255

        blur = cv2.GaussianBlur(float_gray, (0, 0), sigmaX=2, sigmaY=2)
        num = float_gray - blur

        blur = cv2.GaussianBlur(num*num, (0, 0), sigmaX=20, sigmaY=20)
        den = cv2.pow(blur, 0.25)

        gray = num / den

#         invert = cv2.bitwise_not(image)
#         blur = cv2.GaussianBlur(invert , (13,13), 0)
#         inverterblur = cv2.bitwise_not(blur)
#         sketch = cv2.divide(image , inverterblur , scale=256.0)
#         image=cv2.resize(image,(48,48)) #resize the imag
#         img = cv2.Canny(image,100,200)

        images.append(gray)
        labels.append(i)
        counter = counter+1
    
print("total images:",len(images))
print("total images:",len(labels))

In [14]:
encode= LabelEncoder()
labels=encode.fit_transform(labels)
labels

In [15]:
labels=labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
labels=onehotencoder.fit_transform(labels)
labels=labels.toarray()
labels

In [16]:
type(images)

In [17]:
images=np.array(images)
print(images.shape)
print(labels.shape)

In [18]:
images,labels = shuffle(images, labels, random_state=1)


In [19]:
x_train , x_test , y_train , y_test = train_test_split(images, labels, test_size=0.1 , random_state=415)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [20]:
plt.imshow(images[6])
plt.xlabel(labels[6])

In [ ]:
DenseNet201 = DenseNet201(input_shape= (48,48,3) ,  include_top=False , weights="imagenet", pooling='max')


In [ ]:
DenseNet201.trainable = False

In [ ]:
x = Flatten()(DenseNet201.output)


In [ ]:
from tensorflow import keras

prediction = Dense(7, activation='softmax')(x)

# create a model object
model = Model(inputs=DenseNet201.input, outputs=prediction)
opt = keras.optimizers.Adam(learning_rate=0.1)


In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [ ]:
r = model.fit(x_train , y_train , batch_size=16 , epochs=10 , validation_data = (x_test , y_test))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
r = model.fit(x_train , y_train , batch_size=16 , epochs=20 , validation_data = (x_test , y_test))

In [ ]:
r = model.fit(x_train , y_train , batch_size=64 , epochs=10 , validation_data = (x_test , y_test))

In [ ]:
mobile = MobileNet(input_shape= (48,48,3) , dropout = 0.01 ,  include_top=False , weights="imagenet", pooling='max')

In [ ]:
mobile.trainable = False

In [ ]:
x = Flatten()(mobile.output)

In [ ]:
prediction = Dense(7, activation='softmax')(x)

# create a model object
mobile = Model(inputs=mobile.input, outputs=prediction)

In [ ]:
mobile.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
r = mobile.fit(x_train , y_train , batch_size=16 , epochs=40 , validation_data = (x_test , y_test))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
r = model.fit(x_train , y_train , batch_size=32 , epochs=10 , validation_data = (x_test , y_test))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### canny edge detection
1. Noise reduction;
2. Gradient calculation;
3. Non-maximum suppression;
4. Double threshold;
5. Edge Tracking by Hysteresis.

In [ ]:
from scipy.ndimage.filters import convolve

imgs_final = []
img_smoothed = convolve(image, gaussian_kernel(2304, 1))
gradientMat, thetaMat = sobel_filters(img_smoothed)
nonMaxImg = non_max_suppression(gradientMat, thetaMat)
thresholdImg = threshold(nonMaxImg)
img_final = hysteresis(thresholdImg)
imgs_final.append(img_final)

#### Noise reduction:

In [ ]:
def gaussian_kernel(size, sigma=1):
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    normal = 1 / (2.0 * np.pi * sigma**2)
    g =  np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * normal
    return g

In [ ]:
image=cv2.imread("../input/emotion-detection-fer/train/happy/im0.png")
edges = cv2.Canny(image,1,150)
plt.subplot(121),plt.imshow(image,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

In [ ]:
# plt.imshow(imagess)
import cv2
import matplotlib.pyplot as plt

image=cv2.imread("../input/emotion-detection-fer/train/disgusted/im118.png")
# image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
invert = cv2.bitwise_not(image)
blur = cv2.GaussianBlur(invert , (13,13), 0)
inverterblur = cv2.bitwise_not(blur)
sketch = cv2.divide(image , inverterblur , scale=256.0)

plt.imshow(sketch)


In [ ]:
plt.imshow(image)


In [ ]:
import numpy as np
imagesnp=np.array(sketch)
imagesnp.shape

#### Gradient Calculation


In [ ]:
from scipy import ndimage

def sobel_filters(img):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    Ix = ndimage.filters.convolve(img, Kx)
    Iy = ndimage.filters.convolve(img, Ky)
    
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)
    
    return (G, theta)

#### Non-Maximum Suppression


In [ ]:
def non_max_suppression(img, D):
    M, N = img.shape
    Z = np.zeros((M,N), dtype=np.int32)
    angle = D * 180. / np.pi
    angle[angle < 0] += 180

    
    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
                q = 255
                r = 255
                
               #angle 0
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                #angle 45
                elif (22.5 <= angle[i,j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                #angle 90
                elif (67.5 <= angle[i,j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                #angle 135
                elif (112.5 <= angle[i,j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]

                if (img[i,j] >= q) and (img[i,j] >= r):
                    Z[i,j] = img[i,j]
                else:
                    Z[i,j] = 0

            except IndexError as e:
                pass
    
    return Z


#### Double threshold


In [ ]:
def threshold(img, lowThresholdRatio=0.05, highThresholdRatio=0.09):
    
    highThreshold = img.max() * highThresholdRatio;
    lowThreshold = highThreshold * lowThresholdRatio;
    
    M, N = img.shape
    res = np.zeros((M,N), dtype=np.int32)
    
    weak = np.int32(25)
    strong = np.int32(255)
    
    strong_i, strong_j = np.where(img >= highThreshold)
    zeros_i, zeros_j = np.where(img < lowThreshold)
    
    weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))
    
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    
    return (res, weak, strong)

#### Edge Tracking by Hysteresis


In [ ]:
def hysteresis(img, weak=75, strong=255):
    M, N = img.shape  
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img

In [ ]:
import cv2
src = cv2.imread("../input/emotion-detection-fer/train/angry/im1008.png")
src = cv2.GaussianBlur(src, (11, 21),0)
# src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
dst = cv2.Laplacian(src_gray, cv2.CV_8, ksize=5)

In [ ]:
plt.imshow(dst)
dst=np.array(dst)

In [ ]:
dst.shape

In [ ]:
vgg16 = VGG16(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')


In [ ]:
vgg16.trainable = False

In [ ]:
x = Flatten()(vgg16.output)

In [ ]:
prediction = Dense(7, activation='softmax')(x)

# create a model object
vgg16 = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
vgg16.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
r = vgg16.fit(x_train , y_train , batch_size=16 , epochs=40 , validation_data = (x_test , y_test))

In [ ]:
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
model = tf.keras.Sequential()

model.add(Conv2D(16,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,3)))
model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,3)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.01))


model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.2))


model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.01))

# # model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
# # model.add(Activation('relu'))
# # model.add(BatchNormalization())
# # model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
# # model.add(Activation('relu'))
# # model.add(BatchNormalization())
# # model.add(MaxPooling2D(pool_size=(2,2)))
# # model.add(Dropout(0.2))


# # model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
# # model.add(Activation('relu'))
# # model.add(BatchNormalization())
# # model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
# # model.add(Activation('relu'))
# # model.add(BatchNormalization())
# # model.add(MaxPooling2D(pool_size=(2,2)))
# # model.add(Dropout(0.2))


# model.add(Flatten())
# model.add(Dense(64,kernel_initializer='he_normal'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))


# model.add(Dense(64,kernel_initializer='he_normal'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))


model.add(Dense(7,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

model.summary()

In [2]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(48, 48, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
model.summary()

In [23]:
from tensorflow.keras.optimizers import RMSprop
 
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.3),loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
history = model.fit(x_train , y_train , batch_size=16 , epochs=50 , validation_data = (x_test , y_test))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()